In [ ]:
import pandas as pd
import requests as re
from io import BytesIO
from os import path, makedirs
from unidecode import unidecode

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## ObservaSampa

Como vamos usar vários indicadores do ObservaSampa parece fazer sentido usar uma função genérica para download de indicadores do ObservaSampa.

In [ ]:
indicadores_observa = [
    {
        "nome_completo": "11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais",
        "nome_curto": "indicador_110102",
        "id_observa": 283,
        "nome_coluna_valor": "qtd_familias"
    },

    {
        "nome_completo": "11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários",
        "nome_curto": "indicador_110106",
        "id_observa": 402,
        "nome_coluna_valor": "qtd_familias"
    }, 

    {
        "nome_completo": "Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia",
        "nome_curto": "indicador_tpu",
        "id_observa": 662,
        "nome_coluna_valor": "qtd_termos"
    },
    {
        "nome_completo": "Produção de habitação de interesse social",
        "nome_curto": "indicador_producao",
        "id_observa": 285,
        "nome_coluna_valor": "qtd_unidades"
    },
]

In [ ]:
def extrair_observa(id_observa:int) -> pd.DataFrame:
    url_observa = f'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador={id_observa}'
    df_observa = pd.read_csv(url_observa, sep=';', decimal=',', encoding='utf8')
    return df_observa

In [ ]:

def padronizar_subprefeitura(regiao: str) -> str:
    if pd.isna(regiao): return 'Sem informação'
    return unidecode(regiao).strip().lower()

In [ ]:
def tratar_indicador_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Subprefeitura']
    return df

In [ ]:
def tratar_indicador_nao_regionalizado(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['nivel_regional'] == 'Municipio']
    df = df.drop(columns=['nivel_regional', 'região'])
    return df

In [ ]:
def tratar_anos(df: pd.DataFrame,
                nome_coluna_valor:str,
                anos: list[str]=['2022', '2023', '2024']) -> pd.DataFrame:
    df = df.melt(
        ['região', 'indicador'],
        [col for col in df.columns if col in anos],
        var_name='ano',
        value_name=nome_coluna_valor
    )
    return df

In [ ]:
def pipeline_completa(indicador: str) -> pd.DataFrame:
    indicador_info = next((item for item in indicadores_observa if item['nome_curto'] == indicador), None)
    
    df_bom = extrair_observa(indicador_info['id_observa'])

    if df_bom['nivel_regional'].str.lower().str.contains('subprefeitura').any():
        df_bom = tratar_indicador_regionalizado(df_bom)
    else:
        df_bom = tratar_indicador_nao_regionalizado(df_bom)

    df_bom = tratar_anos(df_bom, nome_coluna_valor=indicador_info['nome_coluna_valor'])

    return df_bom

### 11.01.02 Número de famílias beneficiadas por procedimentos de regularização fundiária em núcleos urbanos informais


In [ ]:

df_110102 = pipeline_completa('indicador_110102')
df_110102.head()

### 11.01.06 Número de famílias beneficiadas com obras de urbanização de assentamentos precários

### Produção de habitação de interesse social

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

In [ ]:
url_his = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285'
df_his = pd.read_csv(
    url_his,
    sep=';',
    decimal=',',
    encoding='utf8'
)
df_his

In [ ]:
df_his = df_his[df_his['nivel_regional'] == 'Subprefeitura']
df_his

In [ ]:
df_his = df_his.loc[:, [
    'região',
    'indicador',
    '2022',
    '2023'
]]
df_his

In [ ]:
df_his = df_his.melt(
    ['região', 'indicador'],
    ['2022', '2023'],
    var_name='ano',
    value_name='qtd_unidades'
)
df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [ ]:
url_tpu = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=662'
df_tpu = pd.read_csv(
    url_tpu,
    sep=';',
    decimal=',',
    encoding='utf8'
)
df_tpu

In [ ]:
df_tpu = df_tpu.loc[:, [
    'indicador',
    '2022',
    '2023'
]]
df_tpu

In [ ]:
df_tpu = df_tpu.melt(
    ['indicador'],
    ['2022', '2023'],
    var_name='ano',
    value_name='qtd_termos'
)
df_tpu

In [ ]:
url = "https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285"
df_110107 = pd.read_csv(url, sep=";", decimal=",", encoding="utf-8")
df_110107 = df_110107[df_110107["nivel_regional"] == "Subprefeitura"]
df_110107 = df_110107[["região", "2024"]]
df_110107 = df_110107.rename(columns={
    "região": "Subprefeitura",
    "2024": "Domicílios em favelas (2024)"
})
df_110107["ano"] = 2024
df_110107["Domicílios em favelas (2024)"] = df_110107["Domicílios em favelas (2024)"].astype(int)
df_110107

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [ ]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

for nome, df in [
    ('his_entregue_original', df_his),
    ('tpu_emitido_original', df_tpu),
    ('indicador_11_01_07_2024', df_110107),
    ('indicador_110102', df_110102),
]:
    filename = path.join(output_dir, nome)
    df.to_csv(f'{filename}.csv', sep=';', decimal=',', encoding='utf8', index=False)